In [100]:
%matplotlib inline

import wikipedia
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation

In [93]:
#because my version of pandas is different than the version that descriptions.pkl was created with
with open("descriptions.txt") as desc:
    descriptions = desc.readlines()
titles = []
descs = []
tempDesc = ''
for description in descriptions:
    if description[0].isnumeric():
        titles.append(description[4:-1])
    elif description == '\n':
        descs.append(tempDesc)
        tempDesc = ''
    else:
        tempDesc += description[:-1]
descs.append(tempDesc)

In [94]:
#get rid of NT (National Theatre) in front of some of the names
titles = [t[4:] if t[:3]=="NT -" else t for t in titles]

In [95]:
#augment current descriptions with information from wikipedia
for i in range(len(titles)):
    searchResults = wikipedia.search(titles[i])
    
    #use first search result
    if(len(searchResults) > 0):
        try:
            pageContent = wikipedia.page(searchResults[0]).content
            print(titles[i],'*',searchResults[0])
        except wikipedia.exceptions.DisambiguationError as e:
            try: #TODO: something really weird going on with a few cases of multiple disambiguation errors
                pageContent = wikipedia.page(e.options[0]).content
                print(titles[i],'*',e.options[0])
            except:
                ;#Alright, give up
            
        descs[i] += pageContent

The Infernal Comedy * Musica Angelica Baroque Orchestra
Yuja Wang, piano * Yuja Wang discography
London Philharmonic * London Philharmonic Orchestra
Hamburg Symphony * Hamburger Symphoniker
Chicago Symphony Orc * Gary Gygax
San Francisco Sympho * Boston University Tanglewood Institute
Pavel Haas Quartet * Pavel Haas Quartet
Ballet Preljocaj * Angelin Preljocaj
Joshua Bell * Joshua Bell
Murray Perahia * Murray Perahia
Mariinsky Orchestra * Mariinsky Theatre Orchestra
Detroit Symphony Orc * The Accidentals
New York Philharmoni * Alan Walker (musicologist)
Artemis Quartet * Artemis Quartet
Anne-Sophie Mutter,  * Anne-Sophie Mutter
Bobby McFerrin * Bobby McFerrin
Alison Balsom * Alison Balsom
SITI: Trojan Women * Anne Bogart
Audra McDonald * Audra McDonald
Andras Schiff, piano * Piano Sonata in F minor, D 625 (Schubert)
Apollo's Fire * Apollo's Fire
Joshua Bell, violin * Joshua Bell
Israel Philharmonic * Israel Philharmonic Orchestra
Akademie fur Alte Mu * List of art museums
Olga Kern, pi

//anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [99]:
# Extract bigram matrix
vect = CountVectorizer(ngram_range=(1, 1), stop_words = ENGLISH_STOP_WORDS, min_df = 5)
X = vect.fit_transform(descs)

print(X.shape)

(144, 3605)


In [103]:
#LDA
lda = LatentDirichletAllocation(n_topics=10, max_iter=10)
y = lda.fit_transform(X)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print

print_top_words(lda, vect.get_feature_names(), 20)

Topic #0:
major piano orchestra flat minor concerto op conductor denis clarinet rochester music mozart string 12 symphony sonata quintet philharmonic dances
Topic #1:
series news clock episodes comedy lloyd episode broadcast smith time bbc sketches harmony cast released television jones john second fourth
Topic #2:
museum san art del santa la los el city arts national las new nebraska gallery mexico state francisco building york
Topic #3:
quartet music string choir op violin piano king group quartets beethoven singers works award horn years recording choral international concerto
Topic #4:
singer album music blake released song new american band december july year love september february january jazz time october april
Topic #5:
orchestra music new jazz school 2006 ensemble brass 2010 symphony 2005 york high 2004 2008 2012 concert 2014 2009 2013
Topic #6:
college regular yards des miami cleveland 000 1985 pick prix career island ohio football league bowl championship season years passi